In [1]:
%cd ..

/home/zaccharie/workspace/understanding-unets


In [2]:
%load_ext autoreload
%autoreload 2
import os.path as op
import time

from keras.callbacks import TensorBoard, ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback
from tensorflow import set_random_seed
from tqdm import tqdm_notebook

from data import keras_im_generator, im_generator_BSD68
from evaluate import psnr, ssim
from unet import unet

Using TensorFlow backend.


In [3]:
set_random_seed(1)

In [4]:
batch_size = 32
noise_std = 30 / 255
source = 'bsd68'
validation_split = 0.1
if 'cifar' in source:
    n_samples_train = 5*1e4
    size = 32
elif 'mnist' in source:
    n_samples_train = 6*1e4
    size = 28
elif source == 'bsd68':
    bsd_dir_train = 'BSDS300/images/train'
    n_samples_train = 200
    size = 512
if source in ['cifar', 'cifar_grey', 'mnist']:
    im_gen_train = keras_im_generator(mode='training', validation_split=0.1, batch_size=batch_size, source=source, noise_std=noise_std)
    im_gen_val = keras_im_generator(mode='validation', validation_split=0.1, batch_size=batch_size, source=source, noise_std=noise_std)
elif source == 'bsd68':
    im_gen_train = im_generator_BSD68(path=bsd_dir_train, mode='training', validation_split=0.1, batch_size=batch_size, noise_std=noise_std)
    im_gen_val = im_generator_BSD68(path=bsd_dir_train, mode='validation', validation_split=0.1, batch_size=batch_size, noise_std=noise_std)

In [5]:
run_params = {
#     'n_layers': 4, 
    'n_layers': 2, 
    'pool': 'max', 
#     "layers_n_channels": [16, 32, 64, 128], 
    "layers_n_channels": [16, 32], 
    'layers_n_non_lins': 2,
    'non_relu_contract': False,
}
n_epochs = 50
run_id = f'unet_{source}_{int(noise_std * 255)}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

unet_bsd68_30_1570118181


In [6]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=1000, save_weights_only=False)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

In [10]:
model = unet(input_size=(size, size, 3), lr=1e-3, **run_params)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 512, 512, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 512, 512, 16) 2320        conv2d_10[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 256, 256, 16) 0           conv2d_11[0][0]                  
__________________________________________________________________________________________________
conv2d_12 

In [11]:
# model.fit_generator(
#     im_gen_train, 
#     steps_per_epoch=int((1-validation_split) * n_samples_train / batch_size), 
#     epochs=n_epochs,
#     validation_data=im_gen_val,
#     validation_steps=int(validation_split * n_samples_train / batch_size),
#     verbose=0,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback,],
# #     max_queue_size=100,
#     use_multiprocessing=True,
#     workers=35,
# )

In [12]:
# overfitting trials

data = next(im_gen_train)
val_data = next(im_gen_val)
model.fit(
    x=data[0], 
    y=data[1], 
#     validation_data=val_data, 
    batch_size=data[0].shape[0], 
    epochs=500, 
    verbose=2, 
    shuffle=False,
)
print('Original metrics')
print(psnr(*data))

Instructions for updating:
Use tf.cast instead.
Epoch 1/500


KeyboardInterrupt: 